# Catalogue data analysis for M29, M39 and NGC 6633

Download the data from vizier, GAIA catalogue, with radius 5 arcmin around the center of M29, and filter the data according to paralax

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv("asu-csv.csv", sep=';')
useful_columns = ["_r", "RA_ICRS", "DE_ICRS", "Plx", "e_Plx", "Gmag", "e_Gmag", "BPmag", "e_BPmag", "RPmag", "e_RPmag"]
data = data[useful_columns]

# data = pd.DataFrame([pd.to_numeric(data[col]) for col in data.columns], errors="coerce") 
def to_float(x): 
    try: 
        return float(x)
    except Exception:
        return np.nan 
data = data.applymap(to_float).astype(float)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596 entries, 0 to 1595
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   _r       1596 non-null   float64
 1   RA_ICRS  1596 non-null   float64
 2   DE_ICRS  1596 non-null   float64
 3   Plx      1381 non-null   float64
 4   e_Plx    1381 non-null   float64
 5   Gmag     1585 non-null   float64
 6   e_Gmag   1585 non-null   float64
 7   BPmag    1501 non-null   float64
 8   e_BPmag  1501 non-null   float64
 9   RPmag    1534 non-null   float64
 10  e_RPmag  1534 non-null   float64
dtypes: float64(11)
memory usage: 137.3 KB
None


## Filter stars which have no parallax data

In [4]:
data.dropna(subset=["Plx", "Gmag", "BPmag", "RPmag"], inplace=True)
# data.dropna(subset=["Gmag", "BPmag", "RPmag"], inplace=True)
# no_plx_data = data.dropna(subset=["Plx"])

distance_from_center = data["_r"]
R_ang = 8 # 7 arcmin is approx the radius of m29
d = 1148 # distance to cluster in parsecs
R = d*R_ang/(60*57.299) # radius of the cluster in pc
print(f"cluster raidus is approx {R} pc")

# data = data[distance_from_center < R_ang]
parallax_min = data["Plx"] - data["e_Plx"]
parallax_max = data["Plx"] + data["e_Plx"]
data = data[(parallax_max/1000 > 1/(d + R)) & (parallax_min/1000 < 1/(d - R))]

print(data.info())

cluster raidus is approx 2.671367155913134 pc
<class 'pandas.core.frame.DataFrame'>
Int64Index: 415 entries, 0 to 1589
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   _r       415 non-null    float64
 1   RA_ICRS  415 non-null    float64
 2   DE_ICRS  415 non-null    float64
 3   Plx      415 non-null    float64
 4   e_Plx    415 non-null    float64
 5   Gmag     415 non-null    float64
 6   e_Gmag   415 non-null    float64
 7   BPmag    415 non-null    float64
 8   e_BPmag  415 non-null    float64
 9   RPmag    415 non-null    float64
 10  e_RPmag  415 non-null    float64
dtypes: float64(11)
memory usage: 38.9 KB
None


In [8]:
%matplotlib qt
fig = plt.figure()
ax = plt.axes(projection="3d")
g_mags = data["Gmag"]
bp_mags = data["BPmag"]
rp_mags = data["RPmag"]

ax.scatter3D(bp_mags, bp_mags - g_mags, bp_mags - rp_mags, s=3, color='red')
# no_plx_gmags = no_plx_data["Gmag"] 
# no_plx_bpmags = no_plx_data["BPmag"]
# no_plx_rpmags = no_plx_data["RPmag"]
# ax.scatter3D(no_plx_bpmags, no_plx_bpmags - no_plx_gmags, no_plx_bpmags - no_plx_rpmags, s=1, color='green')
plt.show()